In [1]:
#importo librerias

import pandas as pd
import numpy as np
import json
import glob
import ast
import gzip

In [2]:
#cargo los datasets que voy a necesitar

user_stats = pd.read_parquet("user_stats.parquet")            #leo estos archivos para
user_total_price = pd.read_parquet("user_total_price.parquet") #la PRIMER funcion

user_review = pd.read_parquet("countreviews.parquet")#leo este archivo para la SEGUNDA funcion

ranking = pd.read_parquet("genre.parquet") #leo este archivo para la TERCER funcion

genero_y_usuario_agrupado = pd.read_parquet("userforgenres.parquet")#leo este archivo para la CUARTA funcion



#### CREACION PRIMER FUNCION
cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [3]:
def userdata(user_id : str):
    user_price_data = user_total_price[user_total_price['user_id'] == user_id]
    user_stats_data = user_stats[user_stats['user_id'] == user_id]
    
    if not user_price_data.empty and not user_stats_data.empty:
        gasto = round(user_price_data['price'].iloc[0], 2)
        recomendacion = user_stats_data['recommend_sum'].iloc[0]
        total_items = user_stats_data['items_count'].iloc[0]
        
        return {
            'Usuario:': user_id,
            'Cantidad de dinero gastado:': gasto,
            'Porcentaje de recomendación:':  round((recomendacion / total_items) * 100, 2),
            'Cantidad de ítems:': total_items
        }
    else:
        return 'Usuario no encontrado'
    
userdata('76561197970982479')

{'Usuario:': '76561197970982479',
 'Cantidad de dinero gastado:': 6497.29,
 'Porcentaje de recomendación:': 100.0,
 'Cantidad de ítems:': 831}

#### CREACION SEGUNDA FUNCION
cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [4]:
def countreviews(inicio: str, final: str):
    mask = (user_review['posted'] >= inicio) & (user_review['posted'] <= final)
    reviews = user_review[mask] #creo una mascara para filtrar las reviews en el rango que elijo

    #usuarios unicos que realizaron reviews 
    usuarios_unicos = reviews['user_id'].nunique()

    #porcentaje recomendacion
    porcentaje_recomendacion = round(reviews['recommend'].sum() / reviews['recommend'].count() * 100,2)

    return {
        'Cantidad de usuarios con reviews:': usuarios_unicos,
        'Porcentaje de recomendación:': round(porcentaje_recomendacion, 2)
    }

countreviews('2011-11-11','2014-12-18')

{'Cantidad de usuarios con reviews:': 16146,
 'Porcentaje de recomendación:': 92.39}

#### CREACION TERCER FUNCION
devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [5]:
def genre(genero: str):
    posicion = ranking[ranking['genres'] == genero].index[0] + 1  #busco el genero en el df y su posicion
    return f"Eel genre {genero} se encuentra en el puesto {posicion}"

genre('RPG')

'Eel genre RPG se encuentra en el puesto 13'

#### CREACION CUARTA FUNCION
top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [6]:
def userforgenre(genre: str):
    genero_filtrado = genero_y_usuario_agrupado[genero_y_usuario_agrupado['genres'] == genre]
    top_usuario = genero_filtrado.sort_values(by='playtime_forever', ascending=False).head(5)
    return top_usuario[['user_id', 'user_url', 'playtime_forever']]

userforgenre('Action')

,user_id,user_url,playtime_forever
50657,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307.0
64497,shinomegami,http://steamcommunity.com/id/shinomegami,1580428.0
49457,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212.0
51237,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742.0
44133,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193.0


#### CREACION QUINTA FUNCION
Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

#### CREACION SEXTA FUNCION
según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
